# SGLang profiling数据采集与分析

介绍：本练习旨在实践SGLang的性能剖析数据采集流程，并学习如何分析所采集的剖析数据。采集过程主要包括：数据下载、镜像拉取、容器创建、运行推理任务以及数据导入。分析部分将基于Qwen2.5-7B-Instruct模型的剖析数据展开，具体包括Python层与GPU层的时序图细节分析。

相关文章：[SGLang Profiling数据采集与分析入门](https://zhuanlan.zhihu.com/p/2004605638760763526)

Author: kaiyuan

Email: kyxie@zju.edu.cn

## 1 准备工作

### 1.1 模型下载

从hugging face下载模型到服务器本地盘。模型文件地址：
https://huggingface.co/Qwen/Qwen2-7B-Instruct

设置下载站点：
```
export HF_ENDPOINT=https://hf-mirror.com
```

下载脚本（python）：

```
from huggingface_hub import snapshot_download

repo_id = "Qwen/Qwen2.5-7B-Instruct"
local_dir = "./models/Qwen2.5-7B-Instruct"

# 重试下载，并允许从断点续传
local_dir = snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    local_dir_use_symlinks=False,  # 避免使用符号链接，有时更稳定
    resume_download=True,      # 关键参数：尝试恢复中断的下载
    force_download=False,      # 不强制重新下载已存在的文件
)

print(f"模型已完整下载到：{local_dir}")
```
注：huggingface_hub 的版本 1.4.1


### 1.2 环境创建

推荐使用nvidia的预置镜像，能够避免因环境问题产生error。

镜像拉取：
```
docker pull nvcr.io/nvidia/sglang:26.01-py3
```
版本介绍：[Link](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/sglang?version=26.01-py3)

容器创建示例：
```
docker run -itd --rm --gpus all --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 \
-v /data/nfs/kaiyuan:/data/nfs/kaiyuan \
--name sglang-dev nvcr.io/nvidia/sglang:26.01-py3 bash
```


登入容器：

```
docker exec -it sglang-dev bash
```

测试是否正常：

```
python -c "import torch; import sglang; print(torch.cuda.is_available())"
```

注意：镜像要求NVIDIA驱动版本 >= 570


本例测试机器信息：
- NVIDIA A100-SXM4-80GB
- NVIDIA-SMI 570.172.08
- Driver Version: 570.172.08
- CUDA Version: 13.1


## 2 数据采集

在SGLang官方blog中介绍几种采集方式：[Benchmark and Profiling](https://docs.sglang.io/developer_guide/benchmark_and_profiling.html#benchmark-and-profiling).

选取“HTTP API endpoints”方式实践。

一共需要启动两个console终端(terminal)界面。

- 终端1：启动服务器；
- 终端2：运行客户端脚本。

### 2.1 服务器启动

单机单卡：
```
# 配置profiling导出位置(必须)：
export SGLANG_TORCH_PROFILER_DIR=/data/kaiyuan/llm_infer/profiles

# 启动服务器
python -m sglang.launch_server --model-path /data/kaiyuan/models/Qwen2.5-7B-Instruct

```

单机多卡：

```
SGLANG_TORCH_PROFILER_DIR="/data/kaiyuan/llm_infer/profiles" \
python -m sglang.launch_server \
  --model-path /data/kaiyuan/models/Qwen2.5-7B-Instruct \
  --host 127.0.0.1 \
  --tp-size 4 \
  --port 30000

```

单机多卡（关闭图模式）：

```
SGLANG_CUDA_GRAPH_MODE=0 \
SGLANG_CACHE_GRAPH=0 \
CUDA_LAUNCH_BLOCKING=1 \
SGLANG_TORCH_PROFILER_DIR="/data/kaiyuan/llm_infer/profiles" \
python -m sglang.launch_server \
  --model-path /data/kaiyuan/models/Qwen2.5-7B-Instruct \
  --host 127.0.0.1 \
  --tp-size 4 \
  --port 30000

```
注：
* SGLANG_CUDA_GRAPH_MODE环境变量控制CUDA图模式，0表示关闭；
* tp-size大于1时，会调用多张GPU。
* --model-path 配置下载好的模型地址

### 2.2 客户端

步骤：
- 开启profiling采集；
- 向服务器发送连续5个请求；
- 关闭profiling采集。

profiling采集请求endpoints的基本使用：

- 开启：http://127.0.0.1:30000/start_profile
- 关闭：http://127.0.0.1:30000/stop_profile

采集参数配置：

```
# Start profiling immediately for 10 steps
curl -X POST http://127.0.0.1:30000/start_profile \
  -H "Content-Type: application/json" \
  -d '{
    "num_steps": 10
  }'
```

- output_dir(optional): 性能分析追踪结果的保存目录。若未指定，则使用SGLANG_TORCH_PROFILER_DIR环境变量，或默认使用/tmp目录

- num_steps(optional): 进行性能分析的步数。若未指定，则持续进行分析直至通过/end_profile指令手动停止

- start_step(optional): 开始性能分析的起始步数（包含该步）。适用于跳过预热迭代阶段

- activities(optional): 需要采集的时序图，例如["CPU"]表示仅采集CPU侧数据。默认为["CPU", "GPU"]

- merge_profiles(optional): 是否合并分布式追踪结果。默认为false"


采集过程的python简化版本：

In [ ]:
# -*- coding: gbk -*-
import requests
import time

base_url = "http://127.0.0.1:30000"

# 开始性能分析
requests.post(f"{base_url}/start_profile", timeout=5)

# 发送5个推理请求
for i in range(5):
    requests.post(
        f"{base_url}/v1/completions",
        json={
            "model": "default",
            "prompt": f"测试请求 {i+1}: 请解释人工智能的基本概念",
            "max_tokens": 30,
            "temperature": 0.1
        },
        timeout=15
    )
    time.sleep(0.5)

# 等待并停止性能分析
time.sleep(5)
requests.post(f"{base_url}/stop_profile")



增强版（包含故障判断）：

In [ ]:

import requests
import time
import threading

base_url = "http://127.0.0.1:30000"

def send_stop_profile_with_timeout(timeout=30):
    """发送stop_profile请求，设置超时"""
    try:
        print(f"发送/stop_profile请求，超时设置为{timeout}秒...")
        response = requests.post(f"{base_url}/stop_profile", timeout=timeout)
        print(f"/stop_profile完成: 状态码 {response.status_code}")
        return True
    except requests.exceptions.Timeout:
        print(f"stop_profile超时（{timeout}秒），但可能仍在后台处理")
        print("这通常是正常的，trace文件可能仍在写入")
        return True  # 即使超时也视为成功
    except Exception as e:
        print(f"stop_profile错误: {e}")
        return False

def check_server_status():
    """检查服务器是否仍在运行"""
    try:
        resp = requests.get(f"{base_url}/health", timeout=2)
        return resp.status_code == 200
    except:
        return False

print(">>> Starting profiling session...")
try:
    resp = requests.post(f"{base_url}/start_profile", timeout=5)
    print(f"/start_profile: 状态码 {resp.status_code}")
except Exception as e:
    print(f"/start_profile错误: {e}")

# 执行推理测试
print("\n>>> 运行推理测试（5个请求）...")
for i in range(5):
    print(f"  请求 {i+1}/5")
    try:
        resp = requests.post(
            f"{base_url}/v1/completions",
            json={
                "model": "default",
                "prompt": f"测试请求 {i+1}: 请解释人工智能的基本概念",
                "max_tokens": 30,
                "temperature": 0.1
            },
            timeout=15
        )
        print(f"    状态码: {resp.status_code}")
    except Exception as e:
        print(f"    错误: {e}")
    time.sleep(0.5)

print("\n>>> 等待所有推理请求完成（5秒）...")
time.sleep(5)
print("\n>>> Stopping profiling...")
# 发送stop_profile，设置30秒超时
success = send_stop_profile_with_timeout(timeout=30)

## 3 输出示例

### 3.1 服务器端输出日志

TP size = 1

```
[2026-01-01 08:50:18] INFO:     127.0.0.1:59572 - "POST /start_profile HTTP/1.1" 200 OK
[2026-01-01 08:50:18] Prefill batch, #new-seq: 1, #new-token: 11, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 08:50:18] INFO:     127.0.0.1:59574 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 08:50:19] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 08:50:19] Decode batch, #running-req: 1, #token: 14, token usage: 0.00, cuda graph: True, gen throughput (token/s): 1.99, #queue-req: 0,
[2026-01-01 08:50:19] INFO:     127.0.0.1:59580 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 08:50:20] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 08:50:20] Decode batch, #running-req: 1, #token: 24, token usage: 0.00, cuda graph: True, gen throughput (token/s): 35.99, #queue-req: 0,
[2026-01-01 08:50:20] INFO:     127.0.0.1:59582 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 08:50:21] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 08:50:21] Decode batch, #running-req: 1, #token: 34, token usage: 0.00, cuda graph: True, gen throughput (token/s): 36.09, #queue-req: 0,
[2026-01-01 08:50:21] INFO:     127.0.0.1:59588 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 08:50:22] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 08:50:22] INFO:     127.0.0.1:59594 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 08:50:28] Stop profiling...
[2026-01-01 08:55:53] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles

```

TP size = 4

```
[2026-01-01 09:19:37] INFO:     127.0.0.1:36936 - "POST /start_profile HTTP/1.1" 200 OK
[2026-01-01 09:19:37 TP0] Prefill batch, #new-seq: 1, #new-token: 11, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 09:19:37] INFO:     127.0.0.1:36940 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 09:19:38 TP0] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 09:19:38 TP0] Decode batch, #running-req: 1, #token: 14, token usage: 0.00, cuda graph: True, gen throughput (token/s): 0.54, #queue-req: 0,
[2026-01-01 09:19:38] INFO:     127.0.0.1:36946 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 09:19:39 TP0] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 09:19:39 TP0] Decode batch, #running-req: 1, #token: 24, token usage: 0.00, cuda graph: True, gen throughput (token/s): 38.68, #queue-req: 0,
[2026-01-01 09:19:39] INFO:     127.0.0.1:36950 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 09:19:40 TP0] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 09:19:40 TP0] Decode batch, #running-req: 1, #token: 34, token usage: 0.00, cuda graph: True, gen throughput (token/s): 38.90, #queue-req: 0,
[2026-01-01 09:19:40] INFO:     127.0.0.1:36956 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 09:19:41 TP0] Prefill batch, #new-seq: 1, #new-token: 8, #cached-token: 3, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 09:19:41] INFO:     127.0.0.1:36958 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 09:19:47 TP1] Stop profiling...
[2026-01-01 09:19:47 TP3] Stop profiling...
[2026-01-01 09:19:47 TP0] Stop profiling...
[2026-01-01 09:19:47 TP2] Stop profiling...
[2026-01-01 09:21:07 TP0] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles
[2026-01-01 09:21:07 TP2] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles
[2026-01-01 09:21:07 TP3] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles
[2026-01-01 09:21:07 TP1] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles

```

### 客户端

```
>>> Starting profiling session...
/start_profile: 状态码 200

>>> 运行推理测试（5个请求）...
  请求 1/5
    状态码: 200
  请求 2/5
    状态码: 200
  请求 3/5
    状态码: 200
  请求 4/5
    状态码: 200
  请求 5/5
    状态码: 200

>>> 等待所有推理请求完成（5秒）...

>>> Stopping profiling...
发送/stop_profile请求，超时设置为30秒...
stop_profile超时（30秒），但可能仍在后台处理
这通常是正常的，trace文件可能仍在写入

```

注：profiling采集的时间通常会大于30s，尤其是用挂载nfs网盘的时候。


## 4 profiling阅读方式

打开网页：[https://ui.perfetto.dev/](https://ui.perfetto.dev/)后导入profiling文件，如：xxxx_trace.json

## 5 多TP融合采集方式

融合多TP profiling数据融合




### 服务器端启动方式

```
SGLANG_TORCH_PROFILER_DIR="/data/kaiyuan/llm_infer/profiles" \
python -m sglang.launch_server \
  --model-path /data/kaiyuan/models/Qwen2.5-7B-Instruct \
  --host 127.0.0.1 \
  --tp-size 4 \
  --port 30000
```

### 客户端运行代码

In [ ]:
# -*- coding: gbk -*-
import requests
import time

base_url = "http://127.0.0.1:30000"

# 开始性能分析
url = "http://127.0.0.1:30000/start_profile"
headers = {"Content-Type": "application/json"}
data = {"merge_profiles": True} # 多TP采集融合

response = requests.post(url, headers=headers, json=data)

# 发送5个推理请求
for i in range(5):
    requests.post(
        f"{base_url}/v1/completions",
        json={
            "model": "default",
            "prompt": f"测试请求 {i+1}: 请解释人工智能的基本概念",
            "max_tokens": 30,
            "temperature": 0.1
        },
        timeout=15
    )
    time.sleep(0.5)

# 等待并停止性能分析
time.sleep(5)
requests.post(f"{base_url}/stop_profile")

日志示例：

```
[2026-01-01 12:53:47 TP0] Profiling starts. Traces will be saved to: /data/kaiyuan/llm_infer/profiles (with profile id: 1770728027.861476)
[2026-01-01 12:53:47 TP3] Profiling starts. Traces will be saved to: /data/kaiyuan/llm_infer/profiles (with profile id: 1770728027.861476)
[2026-01-01 12:53:47 TP1] Profiling starts. Traces will be saved to: /data/kaiyuan/llm_infer/profiles (with profile id: 1770728027.861476)
[2026-01-01 12:53:47 TP2] Profiling starts. Traces will be saved to: /data/kaiyuan/llm_infer/profiles (with profile id: 1770728027.861476)
[2026-01-01 12:53:47] INFO:     127.0.0.1:47480 - "POST /start_profile HTTP/1.1" 200 OK
[2026-01-01 12:53:47 TP0] Prefill batch, #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 12:53:53 TP0] Decode batch, #running-req: 1, #token: 14, token usage: 0.00, cuda graph: True, gen throughput (token/s): 0.03, #queue-req: 0,
[2026-01-01 12:53:53] INFO:     127.0.0.1:47482 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 12:53:54 TP0] Prefill batch, #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 12:53:54 TP0] Decode batch, #running-req: 1, #token: 24, token usage: 0.00, cuda graph: True, gen throughput (token/s): 38.91, #queue-req: 0,
[2026-01-01 12:53:54] INFO:     127.0.0.1:47504 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 12:53:55 TP0] Prefill batch, #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 12:53:55 TP0] Decode batch, #running-req: 1, #token: 34, token usage: 0.00, cuda graph: True, gen throughput (token/s): 39.15, #queue-req: 0,
[2026-01-01 12:53:55] INFO:     127.0.0.1:47510 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 12:53:56 TP0] Prefill batch, #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 12:53:56] INFO:     127.0.0.1:47512 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 12:53:57 TP0] Prefill batch, #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 0, #queue-req: 0,
[2026-01-01 12:53:57 TP0] Decode batch, #running-req: 1, #token: 14, token usage: 0.00, cuda graph: True, gen throughput (token/s): 24.92, #queue-req: 0,
[2026-01-01 12:53:57] INFO:     127.0.0.1:47518 - "POST /v1/completions HTTP/1.1" 200 OK
[2026-01-01 12:54:02 TP0] Stop profiling...
[2026-01-01 12:54:02 TP1] Stop profiling...
[2026-01-01 12:54:02 TP2] Stop profiling...
[2026-01-01 12:54:02 TP3] Stop profiling...
[2026-01-01 12:55:25 TP0] Starting profile merge...
[2026-01-01 12:55:25 TP2] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles
[2026-01-01 12:55:25 TP3] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles
[2026-01-01 12:55:25 TP1] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles
[2026-01-01 12:55:25 TP0] Found 4 trace files to merge
[2026-01-01 12:55:25 TP0] Processing /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-0.trace.json.gz with rank info: {'tp_rank': 0}
[2026-01-01 12:55:25 TP0] Processing file: /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-0.trace.json.gz
[2026-01-01 12:55:37 TP0] Processing /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-1.trace.json.gz with rank info: {'tp_rank': 1}
[2026-01-01 12:55:37 TP0] Processing file: /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-1.trace.json.gz
[2026-01-01 12:55:46 TP0] Processing /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-2.trace.json.gz with rank info: {'tp_rank': 2}
[2026-01-01 12:55:46 TP0] Processing file: /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-2.trace.json.gz
[2026-01-01 12:55:56 TP0] Processing /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-3.trace.json.gz with rank info: {'tp_rank': 3}
[2026-01-01 12:55:56 TP0] Processing file: /data/kaiyuan/llm_infer/profiles/1770728027.861476-TP-3.trace.json.gz

[2026-01-01 12:58:58 TP0] Merged profile saved to: /data/kaiyuan/llm_infer/profiles/merged-1770728027.861476.trace.json.gz
[2026-01-01 12:58:58 TP0] Total events merged: 11548182
[2026-01-01 12:59:42 TP0] Profile merge completed: /data/kaiyuan/llm_infer/profiles/merged-1770728027.861476.trace.json.gz
[2026-01-01 12:59:42 TP0] Profiling done. Traces are saved to: /data/kaiyuan/llm_infer/profiles Merged trace: /data/kaiyuan/llm_infer/profiles/merged-1770728027.861476.trace.json.gz (Events: 11548182, Files: 4)

```